In [1]:
import pandas as pd
import spacy
import re
import numpy as np


/home/lazydart/miniconda3/envs/tensorflow/lib/python3.11/site-packages/torch/cuda/__init__.py:628: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/lazydart/miniconda3/envs/tensorflow/lib/python3.11/site-packages/torch/cuda/__init__.py:758: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() if nvml_count < 0 else nvml_count


In [2]:
df = pd.read_pickle('data/typos_annotated.pkl')

In [3]:
typo_df = df[df["Typo Data"].transform(len) > 0].copy()

In [4]:
typo_df["review_text"].iloc[0]

Lekkie i jak jesteś na kuligu to szypko spadasz slabe sanki. NIE POLECAM!!!!!!!!

In [5]:
typo_df.loc[13, "review_text"][1].ent_iob_ == "O"
# in typo_df.loc[13, "review_text"].ents

True

In [6]:
# Temporary fix for the cyrillic characters
typo_df = typo_df[~typo_df["review_text"].astype(str).str.contains("[\u0400-\u04FF]")]

In [7]:
typo_df[typo_df["review_text"].transform(lambda x: len(x.ents)) > 0]

,review_text,sentiment,has_emoji,all_emoji,has_ok,Typo Data
13,"(Bosch, to, już, nie, ta, sama, marka, ,, co, ...",Negative,False,False,True,"[[Bosha, [Boscha, Busha, Bosza, Bosra, Boska, ..."
19,"(Piekarnik, ,, cytuję, serwisanta, elektrolux,...",Negative,False,False,True,"[[elektrolux, [electrolux, elektro lux, elektr..."
20,"(Produkt, ma, inny, kolor, na, panelu, inny, n...",Negative,False,False,False,"[[AVANS, [AWANS, AVANT, EVANS], 38, 208, 5]]"
27,"(Nie, polecam, tego, modelu, lodówki, ., Lodów...",Negative,False,False,False,"[[Humidity, [Humity], 29, 189, 8], [Fresh, [Fr..."
29,"(Kupiłem, za, namową, sąsiadów, ,, którzy, zac...",Negative,False,False,True,"[[Plastikowość, [Plastikowo, Plastikowy], 227,..."
...,...,...,...,...,...,...
174443,"(Dron, idealny, na, start, z, fpv, ,, jednak, ...",Positive,False,False,False,"[[fpv, [fp, fp v], 5, 24, 3]]"
175458,"(Jeszcze, nie, uzywane, ,, mam, Dębica, Frigo,...",Neutral,False,False,False,"[[uzywane, [używane, urywane, wzywane, umywane..."
177647,"(Najgorszy, telefon, jaki, miałam, ., Na, pocz...",Negative,False,False,False,"[[pare, [parę, pera, parr, par, pre, parea, pa..."
177926,"(Ja, niestety, nie, polecam, Foresto, ., 2, ty...",Negative,False,False,False,"[[Foresto, [Forest, Forestom, Foresta, Foresty..."


In [8]:
typo_df["Typo_Data_Exclude_Ner"] = (
    typo_df
        .groupby(typo_df.index)[["review_text", "Typo Data"]]
        .apply(lambda x: # Select only typos that are not named entities
               [typo_info for typo_info in x.iloc[0, 1] if x.iloc[0, 0][typo_info[2]].ent_iob_ == "O"]
               )
)

In [9]:
named_ents = (
    typo_df
        .groupby(typo_df.index)[["review_text", "Typo Data"]]
        .apply(lambda x: # Select only the named entities
               [typo_info for typo_info in x.iloc[0, 1] if x.iloc[0, 0][typo_info[2]].ent_iob_ != "O"]
               )
        .drop_duplicates()
        .iloc[1:]
)

In [10]:
# A lot of False Positives
ne_set = set(named_ents.transform(lambda x: [y[0] for y in x]).sum())

print(len(ne_set))
ne_set
# Maybe rerunning spacy with lowercase might help.

3401


{'KeF750',
 'wyrazniej',
 'Sea',
 'Wrzystko',
 'FHF',
 'Discord',
 'e8',
 'Prosza',
 'DLF',
 'Troxerutin',
 'VSLAM',
 'Purizon',
 'Termetowi',
 'Myślam',
 'Jaroslaw',
 "BOSCH'em",
 'synthese',
 'tubeńka',
 'where',
 'Rhodesian',
 'Stadlerowi',
 'ALE‼️',
 'Womanity',
 'INKWORLD',
 'Macgyvera',
 'aktoirska',
 'jąw',
 'narzekal',
 'Contratubex',
 'KOTLYKATOWICE',
 'Rowniez',
 'PERFUMESCO.pl',
 'Seductive',
 'Melitte',
 'Warriora',
 'trzeszczacy',
 'Dermacolu',
 'bAttlefielda',
 'Elixiru',
 'Professionel',
 'smartboksa',
 'covida',
 'Tracerów',
 'aptx',
 "PRL'u",
 'Dre',
 'Sephorze',
 'Kiddy',
 'Krups',
 'Gio',
 'iPKO',
 'INTENSE',
 'nevím',
 'WYMODEROWANO',
 'Cardioargin',
 'BTS',
 'Frisco',
 'jogfvbnm',
 'Shure',
 'Codemasters',
 'Twinings',
 'Całkowice',
 'Flugger',
 'dziala',
 'HDMI1',
 'Surpresso',
 'Wapteka',
 'Acces',
 'spowalniaja',
 'AC86U',
 'Incandenssence',
 'Paji',
 'motulu',
 'Blutooth',
 'wystacza',
 'chela',
 'Dzwig',
 'kamyczkow',
 'Tokiną',
 'ąeby',
 'spelnila',
 'Winters